In [11]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
import time
from operator import itemgetter

os.environ['SPOTIPY_CLIENT_ID'] = '0dd677ab735f4fd1b9dbf6b236350ba1'#'32b098e058c547168c94a4fdc5dfeeba'
os.environ['SPOTIPY_CLIENT_SECRET'] = 'bbe8736a14ba4e64bfb2d4103c8957aa'#'fb1b59216a104af69e004a30fb535612'
os.environ['SPOTIPY_REDIRECT_URI'] = 'http://google.com/'


client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [12]:
from OOSpotify import *

In [24]:
%%timeit 
a = Artist('Michael Jackson').Albums()

0: Scream
1: XSCAPE
2: XSCAPE - Track by Track Commentary
3: Michael
4: Invincible
5: Invincible
6: BLOOD ON THE DANCE FLOOR/ HIStory In The Mix
7: BLOOD ON THE DANCE FLOOR/ HIStory In The Mix
8: HIStory - PAST, PRESENT AND FUTURE - Book I
9: HIStory - PAST, PRESENT AND FUTURE - BOOK I
10: HIStory - PAST, PRESENT AND FUTURE - BOOK I
11: Dangerous
12: Bad (Remastered)
13: Bad 25th Anniversary
14: Thriller
15: Thriller
16: Thriller
17: Thriller
18: Thriller 25 Super Deluxe Edition
19: Off the Wall
20: Forever Michael
21: Forever, Michael
22: Music & Me
23: Music and Me
24: Ben
25: Ben
26: Got To Be There
27: Got To Be There
0: Scream
1: XSCAPE
2: XSCAPE - Track by Track Commentary
3: Michael
4: Invincible
5: Invincible
6: BLOOD ON THE DANCE FLOOR/ HIStory In The Mix
7: BLOOD ON THE DANCE FLOOR/ HIStory In The Mix
8: HIStory - PAST, PRESENT AND FUTURE - Book I
9: HIStory - PAST, PRESENT AND FUTURE - BOOK I
10: HIStory - PAST, PRESENT AND FUTURE - BOOK I
11: Dangerous
12: Bad (Remastered)


In [25]:
import spotipy.util as util
#UserID:
users = {}
users['zach'] = str(121068889)

scope = 'user-library-read user-read-private'
token = util.prompt_for_user_token(users['zach'],scope)
sp = spotipy.Spotify(auth=token)

from OOSpotify import *

In [27]:
%%timeit 
a = Artist('Michael Jackson').Albums()

0: Scream
1: XSCAPE
2: XSCAPE - Track by Track Commentary
3: Michael
4: Invincible
5: Invincible
6: BLOOD ON THE DANCE FLOOR/ HIStory In The Mix
7: BLOOD ON THE DANCE FLOOR/ HIStory In The Mix
8: HIStory - PAST, PRESENT AND FUTURE - Book I
9: HIStory - PAST, PRESENT AND FUTURE - BOOK I
10: HIStory - PAST, PRESENT AND FUTURE - BOOK I
11: Dangerous
12: Bad (Remastered)
13: Bad 25th Anniversary
14: Thriller
15: Thriller
16: Thriller
17: Thriller
18: Thriller 25 Super Deluxe Edition
19: Off the Wall
20: Forever Michael
21: Forever, Michael
22: Music & Me
23: Music and Me
24: Ben
25: Ben
26: Got To Be There
27: Got To Be There
0: Scream
1: XSCAPE
2: XSCAPE - Track by Track Commentary
3: Michael
4: Invincible
5: Invincible
6: BLOOD ON THE DANCE FLOOR/ HIStory In The Mix
7: BLOOD ON THE DANCE FLOOR/ HIStory In The Mix
8: HIStory - PAST, PRESENT AND FUTURE - Book I
9: HIStory - PAST, PRESENT AND FUTURE - BOOK I
10: HIStory - PAST, PRESENT AND FUTURE - BOOK I
11: Dangerous
12: Bad (Remastered)
